### KAFKA Commands<br> ###
Cheatsheet:https://gist.github.com/ursuad/e5b8542024a15e4db601f34906b30bb5<br>Infiniteloop issue: https://github.com/Parsely/pykafka/issues/502 <br> Examples:https://pythonexample.com/code/python-kafka-consumer-from-beginning/



##### Start Zookeeper and Kafka Server
##### before running code 
    - a.make the following empty directories in the root of the project: output , outputpdf, workingdir , sourcefiles
    - b.sourcefiles(add embedding): http://magnitude.plasticity.ai/word2vec+subword/GoogleNews-vectors-negative300.magnitude 
    - c. install and load kafka in the root directory of project (you shold see kafka_tools)
    - d. change all directory paths references in nn_config to reflect local enviornment 


In [ ]:
###1.Start Scrapyd in root directoryof project:/home/docadmin/Mueller/DTCC_Img/DTCC-demo
scrapyd 

###2.launch scrapyd project in directory where scrapyd srever is active 
scrapyd-deploy 

###3.Start Zookeeper 
%system /home/docadmin/Mueller/D*/D*/kaf*/k*/bin/zookeeper-server-start.sh config/zookeeper.properties

###4.Start Kafka Server 
%system ./bin/kafka-server-start.sh config/server.properties

###5. Lanuch nn_main_flask.py : python3 nn_main_flask.py 
###6. use Curl or http request to make get request to api 

###Shutdown zookeeper 
%system ./bin/zookeeper-server-stop.sh
###Shutdown kafka server 
%system ./bin/kafka-server-stop.sh

#### kafka instance monitoring commands

In [ ]:
#list topics 
./bin/kafka-topics.sh --list --zookeeper localhost:2181
    
#describe a topic 
./bin/kafka-topics.sh --describe --zookeeper localhost:2181 --topic '09-14-18-13-15-09'

##clear topics by changings retention rate
#check retention rate(only will output if retention rate is not set to default)
./bin/kafka-configs.sh --zookeeper localhost:2181 --describe --entity-type topics --entity-name 'bing'    

#change retention to a minute 
./bin/kafka-configs.sh --zookeeper localhost:2181 --alter --entity-type topics --entity-name 'bing' --add-config retention.ms=1000

#reset retention rate to default 
./bin/kafka-configs.sh --zookeeper localhost:2181 --alter --entity-type topics --entity-name 'bing' --delete-config retention.ms
##End process to reset topic 

#total items in a topic 
bin/kafka-run-class.sh kafka.tools.GetOffsetShell --broker-list localhost:9092 --topic '09-14-18-13-15-09' --time -1 --offsets 1 | awk -F ":" '{sum += $3} END {print sum}'

#get earliest offset still in a topic 
./bin/kafka-run-class.sh kafka.tools.GetOffsetShell --broker-list localhost:9092 --topic 'bing' --time -2
    
#get the latest offset in a topic 
./bin/kafka-run-class.sh kafka.tools.GetOffsetShell --broker-list localhost:9092 --topic mytopic --time -1 
    
#delete all topics starting with 09
./bin/kafka-topics.sh --zookeeper localhost:2181 --delete --topic 09-.*

### Kafka parition/topic generation 
- partitions are set to be in excess of consumers to enable process to grow without maintance 

In [ ]:
#AG edit 9/14/18
schedule_settings = {
    'scrapyd' : scrapyd,
    'max_searching_pages' : config.search_results_maxoutput,
    'project_name' : config.project_name,
    'spider_names' : config.spider_names,
    'run_id' : str(datetime.strftime(datetime.now(),'%m-%d-%y-%H-%M-%S')),
    'entity_list' : config.entity_query_list,#config.reg_and_news_entity_list,
    'domain_list': config.domain_list                        
}

entities =  [i[0] for i in schedule_settings['entity_list']]
partition_dict = {'partition_dict' : {entity : TopicPartition(schedule_settings['run_id'], idx) for idx, entity in enumerate(entities)}}
schedule_settings.update(partition_dict)

### Kafka Producer process (Inlayed within scrapyd pipelines) 
Producer() is spun up at the open of each spider, at the close of the spider a consolidated list of all processed items are passed to the kafka broker(que)

In [ ]:
class KafkaProducerPipeline(object):
  def open_spider (self, spider):
        self.topic = spider.topic
        self.partition = spider.partition
            
        self.producer = Producer()
        self.data = []
        
  def close_spider (self, spider):
        self.producer._send_message(topic = self.topic, msg = self.data, partition = self.partition)
        self.producer._close()
  
  def process_item (self, item, spider):
        self.data.append(item)
        return item

### Kafka Consumer process 
#### Multi-threaded processes spun up for each entity after completion of all spiders

In [ ]:
def aggregate_scrapy_output (topic_partition):
    #instantiate consumer to read from the passed topic
    consumer = KafkaConsumer(
                                bootstrap_servers='localhost:9092',
                                auto_offset_reset='earliest',
                                value_deserializer=lambda m: json.loads(m.decode('utf-8'))
                            )
   
    #assign consumer to topic (scrapyd run_id) and partition (entity)
    consumer.assign([topic_partition])
    
    #query kafka broker for data
    data = consumer.poll(timeout_ms=5000)
    #df = pd.DataFrame.from_dict(data[topic_partition][0].value)
    df = pd.DataFrame([item for msg in data[topic_partition][:] for item in msg.value])
    #close consumer
    consumer.close()

    return df

#### THREADING CONSUMER USING process_manager() functionality 

In [ ]:
#start up multi-threaded process manager 
process_manager = Manager()
processes = []

print('P_DICT TYPE: {}, P_DICT VALUE: {}'.format(t, v))

with process_manager as m: 
    dump_list=m.list() 
    unfinished_entities = entities   #unfinished_entities = schedule_settings['entity_list']

    while len(unfinished_entities):
        for idx, entity_name in enumerate(unfinished_entities):
            if entity_jobs_finished(schedule_settings['scrapyd'], schedule_settings['project_name'], entity_name):
                print('Starting postprocessing for entity: {}'.format(unfinished_entities[idx]))
                #p = Process(target = postprocess, args = (schedule_settings['partition_dict'][unfinished_entities.pop(idx)],))
                #p.start()
                p = Process(target = postprocess_wrapper, args = (schedule_settings['partition_dict'][unfinished_entities.pop(idx)], dump_list))
                processes.append(p)
                p.start()

    for process in processes:
                process.join()

    #consolidated_results_df = pd.concat(dump_list)
    consolidated_results_df = []
    consolidated_results_df= consolidated_results_df.append(dump_list)


fpath = 'output/consolidated_results_{}.pk'.format(schedule_settings['run_id'])

with open(fpath, 'wb') as f:
    pickle.dump(consolidated_results_df, f)

print('{time} -- Stored consolidated results dataframe at: {fpath}.'.format(time = str(datetime.strftime(datetime.now(),'%H:%M:%S')), fpath = fpath))

def postprocess_wrapper(topic_partition, lst):
df = postprocess(topic_partition)
lst.append(df)

In [67]:
#a = ["'adfkl'",'adf','adf']
a = ['adfkl','adf','adf']
b = ' '.join(a)
#b = "'"+','.join(a)+"'"
#c=b.replace(',',"','")
#d=[str(c)]

#join(a)
#b=[','.join(a)]str.replace(old, new[, max])

c=b.split(' ')
print(c)
#d=[c]
#list(map(lambda x: '(' +' OR '.join(x)+')',[a]))
#query = [ 'US'+q for q in e]
#query


['adfkl', 'adf', 'adf']


[entity_tuple(Entity_Name='zack c', Entity_State='nc', Employer='EYY'),
 entity_tuple(Entity_Name='jimmy c', Entity_State='nc', Employer='EYY'),
 entity_tuple(Entity_Name='david c', Entity_State='nc', Employer='EYY')]

In [10]:
#identify the named tuple associated with a specific name 
[item.Entity_State for item in zacks if item.Entity_Name == 'zack c']


['nc']

#### E2E Pipeline for Consumption of messages from consumer (BING)

In [7]:
#import modules 
import os
import sys
sys.path.insert(0, "//home//docadmin//ZackC//NegNews_kafka//NN_Kafka//")
os.chdir("/home/docadmin/ZackC/NegNews_kafka/NN_Kafka/")
from scrapyd_driver_v3 import *
from nn_base.nn_textprocess import indv_found
from nn_base.nn_textprocess import rough_date
from nn_base.nn_textprocess import entity_found
from nn_base.nn_textprocess import trim_key
from nn_base.nn_textprocess import stemsims
from nn_base.nn_textprocess import MinHash
import nn_config as config
import pickle
import time
from scrapyd_api import ScrapydAPI
import dateparser
import numpy as np
import pandas as pd
import copy
import pymagnitude
from nn_base.nn_translate import langname_to_iso639
from nn_base.nn_translate import translate_searchterms
from nn_base.nn_textprocess import OrigTxt_PreProcess_DTCC
from nn_base.nn_textprocess import entity_match_score
from nn_base.nn_textprocess import entity_stem_match
from nn_base.nn_textprocess import w2v_SortOutput
from nn_base.nn_textprocess import LSA_function
from nn_base.nn_textprocess import rerank_searchresults
from nn_base.nn_textprocess import summary_scores
from nn_base.nn_textprocess import w2v_se2
from nn_base.nn_classes import search_entity as SE
from nn_base.nn_pdf import create_pdf_summary
import re
import json 
from nltk.corpus import stopwords
import numpy as np
import random
#flask requirments 
from flask import Flask, request, render_template, Response
import sys
import time
import os
#import subprocess
#import nn_config
from scrapyd_api import ScrapydAPI
import json
from fileupload import saveFile
from nn_base.load_source_data import loadExcel, sqlpulldata
from collections import namedtuple
entity_tuple = namedtuple('entity_tuple',['Entity_Name','Entity_State','Employer','IndvOREntity'])

In [25]:
#create named tuple to hold information about entities being run 
entity_query_list=[]
entity_tuples= [('Raymond James','','','1'),('Fifth Third Bank','','','1'),('CitiGroup','','','1'),('UBS','','','1'),('Goldman Sachs','','','1')]
entity_query_list.append([entity_tuple._make(x) for x in entity_tuples])
entity_query_list[0]

test = NamedTupList(entity_tuples)

def NamedTupList(list_of_tups):
    namedtups = []
    entity_tuple = namedtuple('entity_tuple',['Entity_Name','Entity_State','Employer','IndvOREntity'])
    namedtups.append([entity_tuple._make(x) for x in list_of_tups])
    return namedtups[0]

[i.Entity_Name for i in NamedTupList(entity_tuples)]

['Raymond James', 'Fifth Third Bank', 'CitiGroup', 'UBS', 'Goldman Sachs']

#### NNMain

In [ ]:
from nn_postprocessing import postprocess
from nn_schedulespiders import schedule_spiders, entity_jobs_finished, NamedTupList
from kafka import KafkaProducer, KafkaConsumer, TopicPartition
from multiprocessing import Process, Manager
from scrapyd_api import ScrapydAPI
from datetime import datetime
import nn_config as config
import json
import pandas as pd
import pickle 
import timeit 

def main():
    '''
    Pre-conditions:
        Zookeeper server has been started
        Kafka server has been started
        Scrapyd has been started (CLI: scrapyd)
        Scrapyd project has been deployed in project root directory (CLI 'scrapd-deploy')
    '''

    scrapyd = ScrapydAPI('http://localhost:6800')

    #AG edit 9/14/18
    schedule_settings = {
        'scrapyd' : scrapyd,
        'max_searching_pages' : config.search_results_maxoutput,
        'project_name' : config.project_name,
        'spider_names' : config.spider_names,
        'run_id' : str(datetime.strftime(datetime.now(),'%m-%d-%y-%H-%M-%S')),
        'entity_list' : NamedTupList(config.entity_query_list),
        'domain_list': config.domain_list                        
    }

    #partition_dict = {'partition_dict' : {entity : TopicPartition(schedule_settings['run_id'], idx) for idx, entity in enumerate(schedule_settings['entity_list'])}}
    entities =  [i.Entity_Name for i in schedule_settings['entity_list']]
    partition_dict = {'partition_dict' : {entity : TopicPartition(schedule_settings['run_id'], idx) for idx, entity in enumerate(entities)}}
    schedule_settings.update(partition_dict)

    print('ARGUMENT PASSED AS PARTITION DICT: {}'.format(partition_dict['partition_dict']))
    print('SCHEDULE SETTINGS:')
    for k, v in schedule_settings.items():
        print('{}: {}'.format(k, v))

    #schedule_spiders(**schedule_settings)
    t, v = schedule_spiders(
        scrapyd,
        schedule_settings['entity_list'],
        schedule_settings['max_searching_pages'],
        schedule_settings['project_name'],
        schedule_settings['spider_names'],
        schedule_settings['run_id'],
        schedule_settings['partition_dict'],
        schedule_settings['domain_list']
    )

    #start up multi-threaded process manager 
    process_manager = Manager()
    processes = []
    #consolidated_results_df =[]
    print('P_DICT TYPE: {}, P_DICT VALUE: {}'.format(t, v))
    starttime = timeit.timeit()

    with process_manager as m: 
        dump_list=m.list() 
        unfinished_entities = entities   #unfinished_entities = schedule_settings['entity_list']

        while len(unfinished_entities):
            for idx, entity_name in enumerate(unfinished_entities):
                if entity_jobs_finished(schedule_settings['scrapyd'], schedule_settings['project_name'], entity_name):
                    print('Starting postprocessing for entity: {}, at time {}'.format(unfinished_entities[idx],datetime.now()))
                    print('Starting postprocessing for entity: {}'.format(unfinished_entities[idx]))
                    p = Process(target = postprocess_wrapper, args = (schedule_settings['partition_dict'][unfinished_entities.pop(idx)], dump_list))
                    processes.append(p)
                    p.start()
                    #p.join()

        print('{time} -- All postprocessing jobs started. Waiting for completion ..'.format(time = str(datetime.strftime(datetime.now(),'%H:%M:%S'))))
        for process in processes:
            process.join()

        #consolidated_results_df = pd.concat(dump_list)

        #store output from dumplist whch just contains all information about the entities the process was run on 
        consolidated_results_df= dump_list
        print(dump_list)

    fpath = 'output/consolidated_results_{}.pk'.format(schedule_settings['run_id'])
        
    with open(fpath, 'wb') as f:
        pickle.dump(consolidated_results_df, f)
    endtime = timeit.timeit() 
    totaltime = endtime-starttime
    print(totaltime)
    print('{time} -- Stored consolidated results dataframe at: {fpath}.'.format(time = str(datetime.strftime(datetime.now(),'%H:%M:%S')), fpath = fpath))
    
def postprocess_wrapper(topic_partition, lst):
    df = postprocess(topic_partition)
    lst.append(df)

if __name__ == '__main__':
    main()

In [ ]:
def postprocess_wrapper(topic_partition, lst):
    df = postprocess(topic_partition)
    lst.append(df)

(0, 0)

### FLASK API SKELTON